In [1]:
!pip install ultralytics
import os, shutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall:

In [2]:
import os
import shutil
import random

# -------------------------------
# Define paths to the C2A dataset
# -------------------------------
base_path = "/kaggle/input/c2a-dataset/C2A_Dataset/new_dataset3"

train_img_dir = os.path.join(base_path, "train", "images")
train_lbl_dir = os.path.join(base_path, "train", "labels")
val_img_dir   = os.path.join(base_path, "val", "images")
val_lbl_dir   = os.path.join(base_path, "val", "labels")

# ------------------------------------------
# Create working directory (train/val/test)
# ------------------------------------------
work_dir = "/kaggle/working/flood_yolo"

for subdir in [
    "images/train", "images/val", "images/test",
    "labels/train", "labels/val", "labels/test"
]:
    os.makedirs(os.path.join(work_dir, subdir), exist_ok=True)

In [3]:
# -------------------------------
# Filter flood images
# -------------------------------
flood_train_imgs = [
    f for f in os.listdir(train_img_dir)
    if f.startswith("flood_image")
]

flood_val_imgs = [
    f for f in os.listdir(val_img_dir)
    if f.startswith("flood_image")
]

print(
    f"Found {len(flood_train_imgs)} flood training images and "
    f"{len(flood_val_imgs)} flood validation images."
)


# ------------------------------------------
# Split validation → val (80%) + test (20%)
# ------------------------------------------
random.seed(42)  # reproducibility

num_test = max(1, int(0.20 * len(flood_val_imgs)))
test_imgs = random.sample(flood_val_imgs, num_test)
val_imgs_final = list(set(flood_val_imgs) - set(test_imgs))

print(f"Validation images: {len(val_imgs_final)}")
print(f"Test images: {len(test_imgs)}")

# -------------------------------
# Copy training images
# -------------------------------
for img_file in flood_train_imgs:
    src_img = os.path.join(train_img_dir, img_file)
    src_lbl = os.path.join(train_lbl_dir, img_file.replace(".png", ".txt"))

    dst_img = os.path.join(work_dir, "images/train", img_file)
    dst_lbl = os.path.join(work_dir, "labels/train", img_file.replace(".png", ".txt"))

    shutil.copy(src_img, dst_img)
    shutil.copy(src_lbl, dst_lbl)

# -------------------------------
# Copy validation images (80%)
# -------------------------------
for img_file in val_imgs_final:
    src_img = os.path.join(val_img_dir, img_file)
    src_lbl = os.path.join(val_lbl_dir, img_file.replace(".png", ".txt"))

    dst_img = os.path.join(work_dir, "images/val", img_file)
    dst_lbl = os.path.join(work_dir, "labels/val", img_file.replace(".png", ".txt"))

    shutil.copy(src_img, dst_img)
    shutil.copy(src_lbl, dst_lbl)

# -------------------------------
# Copy test images (20%)
# -------------------------------
for img_file in test_imgs:
    src_img = os.path.join(val_img_dir, img_file)
    src_lbl = os.path.join(val_lbl_dir, img_file.replace(".png", ".txt"))

    dst_img = os.path.join(work_dir, "images/test", img_file)
    dst_lbl = os.path.join(work_dir, "labels/test", img_file.replace(".png", ".txt"))

    shutil.copy(src_img, dst_img)
    shutil.copy(src_lbl, dst_lbl)

Found 1529 flood training images and 542 flood validation images.
Validation images: 434
Test images: 108


In [4]:
# -------------------------------
# Create dataset YAML
# -------------------------------
dataset_yaml = f"""
path: {work_dir}
train: images/train
val: images/val
test: images/test
names:
  0: human
"""

yaml_path = os.path.join(work_dir, "flood.yaml")
with open(yaml_path, "w") as f:
    f.write(dataset_yaml)

print("Flood-only dataset YAML created at", yaml_path)

Flood-only dataset YAML created at /kaggle/working/flood_yolo/flood.yaml


In [5]:
from ultralytics import YOLO
# Load pretrained RT-DETR (official HF version)
model = YOLO('rtdetr-l.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
results = model.train(
    data=os.path.join(work_dir, "flood.yaml"),  # use the flood-only dataset
    epochs=100,
    batch=4,
    imgsz=960,
    workers=2,
    lr0=0.01,               # Initial learning rate
    lrf=0.01,               # Final learning rate fraction (end of cosine schedule)
    momentum=0.937,         # Momentum for optimizer
    weight_decay=0.0005,    # Helps regularization
    warmup_epochs=3,        # Warmup for stable initial convergence
    augment=True,
    #optimizer="AdamW",
    patience=25       # early stopping if no improvement
    #close_mosaic=10,  # disable mosaic augmentation after 10 epochs
    #degrees=20,       # augmentation: rotation range
    #translate=0.2,    # augmentation: translation
    #scale=0.5,        # augmentation: scale (zoom) range
    #shear=10,         # augmentation: shear angle
    #flipud=0.1,       # augmentation: vertical flip probability
    #hsv_h=0.015,      # augmentation: HSV hue
    #hsv_s=0.7,        # augmentation: HSV saturation
    #hsv_v=0.4         # augmentation: HSV value (brightness)
)

Ultralytics 8.3.252 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/flood_yolo/flood.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train13, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=25, perspective=0

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      1/100       6.6G     0.9848     0.3883     0.1752         31        960: 100% ━━━━━━━━━━━━ 383/383 1.3it/s 4:51<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 68/68 2.6it/s 25.7s0.4ss
                   all        542      18562      0.821      0.755      0.771      0.475

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      2/100      7.13G     0.7551     0.3819    0.09128         46        960: 100% ━━━━━━━━━━━━ 383/383 1.3it/s 4:46<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 68/68 2.8it/s 24.7s0.4ss
                   all        542      18562      0.834      0.768      0.784      0.494

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      3/100      7.13G       0.66     0.3852    0.05888        207        960: 0% ──────────── 0/383  0.7s

...

     99/100      7.31G     0.4676     0.3286    0.04638         24        960: 100% ━━━━━━━━━━━━ 383/383 1.4it/s 4:40<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 68/68 2.8it/s 24.5s0.4ss
                   all        542      18562      0.892      0.852      0.856       0.62

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
    100/100      7.31G      0.374     0.3162    0.04019        119        960: 0% ──────────── 0/383  0.7s

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


    100/100      7.31G     0.4605     0.3239    0.04648         31        960: 100% ━━━━━━━━━━━━ 383/383 1.4it/s 4:40<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 68/68 2.8it/s 24.5s0.4ss
                   all        542      18562      0.889      0.854      0.858      0.622

100 epochs completed in 8.573 hours.
Optimizer stripped from /kaggle/working/runs/detect/train13/weights/last.pt, 66.3MB
Optimizer stripped from /kaggle/working/runs/detect/train13/weights/best.pt, 66.3MB

Validating /kaggle/working/runs/detect/train13/weights/best.pt...
Ultralytics 8.3.252 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
rt-detr-l summary: 310 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 68/68 2.7it/s 25.4s0.4ss


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        542      18562      0.896       0.85      0.859      0.622
Speed: 0.4ms preprocess, 39.4ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /kaggle/working/runs/detect/train13


In [7]:
#  Get path from results object
best_model_path = os.path.join(results.save_dir, "weights", "best.pt")

# Copy to a fixed known location
shutil.copy(best_model_path, "/kaggle/working/flood_finetuned_best.pt")
print("Finetuned model saved to /kaggle/working/flood_finetuned_best.pt")

Finetuned model saved to /kaggle/working/flood_finetuned_best.pt


In [8]:
# Performance on val set
# -------------------------------
# Load the best model
fv_model = YOLO("/kaggle/working/flood_finetuned_best.pt")

# Validate on the flood val set again
fv_metrics = fv_model.val(data=os.path.join(work_dir, "flood.yaml"), split="val", imgsz=960, batch=4)

# Extract metrics
fv_precision = fv_metrics.box.mp
fv_recall    = fv_metrics.box.mr
fv_map50     = fv_metrics.box.map50
fv_map5095   = fv_metrics.box.map

print(f"Precision: {fv_precision:.4f}")
print(f"Recall: {fv_recall:.4f}")
print(f"mAP@0.5: {fv_map50:.4f}")
print(f"mAP@0.5:0.95: {fv_map5095:.4f}")


Ultralytics 8.3.252 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
rt-detr-l summary: 310 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3304.6±886.1 MB/s, size: 856.0 KB)
val: Scanning /kaggle/working/flood_yolo/labels/val.cache... 542 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 542/542 227.3Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 136/136 5.0it/s 27.0s0.2s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        542      18562      0.895      0.853      0.861      0.637
Speed: 1.7ms preprocess, 43.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /kaggle/working/runs/detect/val4
Precision: 0.8949
Recall: 0.8529
mAP@0.5: 0.8612
mAP@0.5:0.95: 0.6373


In [9]:
import pandas as pd
import os

# ===============================
# Save val metrics to Excel
# ===============================

metrics_dict = {
    "Precision": fv_metrics.box.mp,
    "Recall": fv_metrics.box.mr,
    "mAP@0.5": fv_metrics.box.map50,
    "mAP@0.5:0.95": fv_metrics.box.map,
}

metrics_df = pd.DataFrame([metrics_dict])

# output directory (same work_dir you already use)
metrics_dir = os.path.join(work_dir, "metrics")
os.makedirs(metrics_dir, exist_ok=True)

excel_path = os.path.join(metrics_dir, "rt_detr_val_metrics.xlsx")
metrics_df.to_excel(excel_path, index=False)

print(f"Metrics saved to: {excel_path}")

Metrics saved to: /kaggle/working/flood_yolo/metrics/rt_detr_val_metrics.xlsx


In [10]:
# Performance on test set
# -------------------------------
# Load the best model
ft_model = YOLO("/kaggle/working/flood_finetuned_best.pt")

# Validate on the flood val set again
ft_metrics = ft_model.val(data=os.path.join(work_dir, "flood.yaml"), split="test", imgsz=960, batch=4)

# Extract metrics
ft_precision = ft_metrics.box.mp
ft_recall    = ft_metrics.box.mr
ft_map50     = ft_metrics.box.map50
ft_map5095   = ft_metrics.box.map

print(f"Precision: {ft_precision:.4f}")
print(f"Recall: {ft_recall:.4f}")
print(f"mAP@0.5: {ft_map50:.4f}")
print(f"mAP@0.5:0.95: {ft_map5095:.4f}")


Ultralytics 8.3.252 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
rt-detr-l summary: 310 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2747.1±655.5 MB/s, size: 556.2 KB)
val: Scanning /kaggle/working/flood_yolo/labels/test... 130 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 130/130 1.1Kit/s 0.1s<0.1s
val: New cache created: /kaggle/working/flood_yolo/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 33/33 4.5it/s 7.4s0.2s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        130       4470      0.897      0.856       0.87      0.646
Speed: 2.7ms preprocess, 48.2ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /kaggle/working/runs/detect/val5
Precision: 0.8969
Recall: 0.8563
mAP@0.5: 0.8697
mAP@0.5:0.95: 0.6460


In [11]:
import pandas as pd
import os

# ===============================
# Save test metrics to Excel
# ===============================

metrics_dict = {
    "Precision": ft_metrics.box.mp,
    "Recall": ft_metrics.box.mr,
    "mAP@0.5": ft_metrics.box.map50,
    "mAP@0.5:0.95": ft_metrics.box.map,
}

metrics_df = pd.DataFrame([metrics_dict])

# output directory (same work_dir you already use)
metrics_dir = os.path.join(work_dir, "metrics")
os.makedirs(metrics_dir, exist_ok=True)

excel_path = os.path.join(metrics_dir, "rt_detr_test_metrics.xlsx")
metrics_df.to_excel(excel_path, index=False)

print(f"Metrics saved to: {excel_path}")

Metrics saved to: /kaggle/working/flood_yolo/metrics/rt_detr_test_metrics.xlsx


In [12]:
import matplotlib.pyplot as plt
import os
import cv2

# ==================================
# Save first 20 TEST prediction results
# ==================================

save_pred_dir = os.path.join(work_dir, "predictions", "test")
os.makedirs(save_pred_dir, exist_ok=True)

# Test images directory
test_images_dir = os.path.join(work_dir, "images/test")
test_images = sorted(os.listdir(test_images_dir))[:20]

for idx, img_file in enumerate(test_images):
    img_path = os.path.join(test_images_dir, img_file)

    # Run prediction
    results = model.predict(
        source=img_path,
        conf=0.25,
        save=False,
        verbose=False
    )

    # Render prediction
    rendered_img = results[0].plot()

    # Convert BGR to RGB for matplotlib
    rendered_img = cv2.cvtColor(rendered_img, cv2.COLOR_BGR2RGB)

    # Plot
    plt.figure(figsize=(6, 6))
    plt.imshow(rendered_img)
    plt.axis("off")

    base_name = os.path.splitext(img_file)[0]

    png_path = os.path.join(save_pred_dir, f"{base_name}.png")
    eps_path = os.path.join(save_pred_dir, f"{base_name}.eps")

    plt.savefig(png_path, dpi=300, bbox_inches="tight")
    plt.savefig(eps_path, format="eps", bbox_inches="tight")
    plt.close()

    print(f"[{idx+1}/{len(test_images)}] Saved TEST prediction: {png_path} and {eps_path}")


[1/20] Saved TEST prediction: /kaggle/working/flood_yolo/predictions/test/flood_image0001_2.png and /kaggle/working/flood_yolo/predictions/test/flood_image0001_2.eps
[2/20] Saved TEST prediction: /kaggle/working/flood_yolo/predictions/test/flood_image0001_4.png and /kaggle/working/flood_yolo/predictions/test/flood_image0001_4.eps
[3/20] Saved TEST prediction: /kaggle/working/flood_yolo/predictions/test/flood_image0004_1.png and /kaggle/working/flood_yolo/predictions/test/flood_image0004_1.eps
[4/20] Saved TEST prediction: /kaggle/working/flood_yolo/predictions/test/flood_image0007_2.png and /kaggle/working/flood_yolo/predictions/test/flood_image0007_2.eps
[5/20] Saved TEST prediction: /kaggle/working/flood_yolo/predictions/test/flood_image0009_4.png and /kaggle/working/flood_yolo/predictions/test/flood_image0009_4.eps
[6/20] Saved TEST prediction: /kaggle/working/flood_yolo/predictions/test/flood_image0013_4.png and /kaggle/working/flood_yolo/predictions/test/flood_image0013_4.eps
[7/2